In [ ]:
!mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
!kaggle datasets download -d grassknoted/asl-alphabet

asl-alphabet.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip asl-alphabet.zip -d asl-alphabet

Archive:  asl-alphabet.zip
replace asl-alphabet/asl_alphabet_test/asl_alphabet_test/A_test.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [ ]:
# Load Data
import os
import cv2
import numpy as np

# Data Visualisation
import matplotlib.pyplot as plt

# Model Training
from tensorflow.keras import utils
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import VGG16

# Warning
import warnings
warnings.filterwarnings("ignore")

# Main
import os
import glob
import cv2
import numpy as np
import pandas as pd
import gc
import string
import time
import random
from PIL import Image
from tqdm import tqdm
tqdm.pandas()

# Visualization
import matplotlib
import matplotlib.pyplot as plt




from sklearn.manifold import TSNE

# Model
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array, array_to_img
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from keras.models import load_model, Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
# Configuration
class CFG:
  batch_size = 128
  img_height = 32
  img_width = 32
  epochs = 10
  num_classes = 29
  img_channels = 3

def seed_everything(seed: int):
  random.seed(seed)
  os.environ["PYTHONHASHSEED"] = str(seed)
  np.random.seed(seed)
  tf.random.set_seed(seed)

In [ ]:
# Labels
TRAIN_PATH = "/content/asl-alphabet/asl_alphabet_train/asl_alphabet_train"
labels = []
alphabet = list(string.ascii_uppercase)
labels.extend(alphabet)
labels.extend(["del", "nothing", "space"])
print(labels)

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space']


In [ ]:
# Create Metadata
list_path = []
list_labels = []
for label in labels:
  label_path = os.path.join(TRAIN_PATH, label, "*")
  image_files = glob.glob(label_path)

  sign_label = [label] * len(image_files)

  list_path.extend(image_files)
  list_labels.extend(sign_label)

metadata = pd.DataFrame({
    "image_path": list_path,
    "label": list_labels
})

metadata

,image_path,label
0,/content/asl-alphabet/asl_alphabet_train/asl_a...,A
1,/content/asl-alphabet/asl_alphabet_train/asl_a...,A
2,/content/asl-alphabet/asl_alphabet_train/asl_a...,A
3,/content/asl-alphabet/asl_alphabet_train/asl_a...,A
4,/content/asl-alphabet/asl_alphabet_train/asl_a...,A
...,...,...
86995,/content/asl-alphabet/asl_alphabet_train/asl_a...,space
86996,/content/asl-alphabet/asl_alphabet_train/asl_a...,space
86997,/content/asl-alphabet/asl_alphabet_train/asl_a...,space
86998,/content/asl-alphabet/asl_alphabet_train/asl_a...,space


In [ ]:
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    metadata['image_path'],
    metadata['label'],
    test_size=0.2,
    random_state=2253,
    shuffle=True,
    stratify=metadata['label']
)

# Create a DataFrame for the training set test set
data_train = pd.DataFrame({
    'image_path': X_train,
    'label': y_train
})

data_test = pd.DataFrame({
    'image_path': X_test,
    'label': y_test
})

# Split the training set into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    data_train['image_path'],
    data_train['label'],
    test_size=0.2/0.7,  # Assuming you want 20% for validation out of the training set
    random_state=2253,
    shuffle=True,
    stratify=data_train['label']
)

# Create a DataFrame for the validation set
data_val = pd.DataFrame({
    'image_path': X_val,
    'label': y_val
})

In [ ]:
def data_augmentation():
    datagen = ImageDataGenerator(
        rescale=1/255.,
        # Add other augmentation parameters as needed
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    train_generator = datagen.flow_from_dataframe(
        data_train,
        directory='./',
        x_col='image_path',
        y_col='label',
        class_mode='categorical',
        batch_size=CFG.batch_size,
        target_size=(CFG.img_height, CFG.img_width)
    )

    validation_generator = datagen.flow_from_dataframe(
        data_val,
        directory='./',
        x_col='image_path',
        y_col='label',
        class_mode='categorical',
        batch_size=CFG.batch_size,
        target_size=(CFG.img_height, CFG.img_width)
    )

    test_generator = datagen.flow_from_dataframe(
        data_test,  # Assuming you have a DataFrame for test data
        directory='./',
        x_col='image_path',
        y_col='label',
        class_mode='categorical',
        batch_size=CFG.batch_size,
        target_size=(CFG.img_height, CFG.img_width),
        shuffle=False  # Set to False for test data
    )

    return train_generator, validation_generator, test_generator

# Seed for reproducibility
seed_everything(2253)

# Get the generators
train_generator, validation_generator, test_generator = data_augmentation()

Found 69600 validated image filenames belonging to 29 classes.
Found 19886 validated image filenames belonging to 29 classes.
Found 17400 validated image filenames belonging to 29 classes.


In [ ]:
# Define input shape
input_shape = (32, 32, 3)

# Load the VGG16 model without the top (classification) layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)

# Add your custom classification layers on top of the base model
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(128, activation='relu')(x)  # You can adjust the number of units as needed
predictions = Dense(29, activation='softmax')(x)  # num_classes is the number of classes in your dataset

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Summarize the model architecture
model.summary()


Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 32, 32, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 32, 32, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 16, 16, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 16, 16, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 16, 16, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 8, 8, 128)         0   

In [ ]:
# Compile the model
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Create a ModelCheckpoint callback
checkpoint_callback = ModelCheckpoint(
    filepath='/content/sample_data/best_model_weights.h5',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // CFG.batch_size,
    epochs= CFG.epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // CFG.batch_size,
    callbacks=[checkpoint_callback],
    shuffle=True,
    verbose=1
)

Epoch 1/10
543/543 [==============================] - ETA: 0s - loss: 2.6944 - accuracy: 0.1646
Epoch 1: val_accuracy improved from -inf to 0.32510, saving model to /content/sample_data/best_model_weights.h5
543/543 [==============================] - 150s 267ms/step - loss: 2.6944 - accuracy: 0.1646 - val_loss: 2.0153 - val_accuracy: 0.3251
Epoch 2/10
543/543 [==============================] - ETA: 0s - loss: 1.2996 - accuracy: 0.5487
Epoch 2: val_accuracy improved from 0.32510 to 0.72596, saving model to /content/sample_data/best_model_weights.h5
543/543 [==============================] - 128s 236ms/step - loss: 1.2996 - accuracy: 0.5487 - val_loss: 0.7726 - val_accuracy: 0.7260
Epoch 3/10
543/543 [==============================] - ETA: 0s - loss: 0.6387 - accuracy: 0.7824
Epoch 3: val_accuracy improved from 0.72596 to 0.82203, saving model to /content/sample_data/best_model_weights.h5
543/543 [==============================] - 126s 232ms/step - loss: 0.6387 - accuracy: 0.7824 - val_l

In [ ]:
scores = model.evaluate(test_generator)
print("%s: %2f%%" % ("Evaluate Test Accuracy", scores[1]*100))

136/136 [==============================] - 22s 162ms/step - loss: 0.1752 - accuracy: 0.9532
Evaluate Test Accuracy: 95.321840%


In [ ]:
# Confusion Matrix
fine_tuned_model = load_model("/content/sample_data/best_model_weights.h5")
predictions = fine_tuned_model.predict(test_generator)

# Get the true labels from the generator
true_labels = test_generator.classes

# Compute the confusion matrix using tf.math.confusion_matrix
confusion_matrix = tf.math.confusion_matrix(
    labels = true_labels,
    predictions = predictions.argmax(axis=1),
    num_classes = 29
)

In [ ]:
# Load the saved model
model = tf.keras.models.load_model('/content/sample_data/best_model_weights.h5')

In [ ]:
# Testing with an image
image_path = '/content/asl-alphabet/asl_alphabet_train/asl_alphabet_train/Y/Y10.jpg'
img = cv2.imread(image_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = cv2.resize(img,(32,32))

img = tf.keras.applications.mobilenet_v2.preprocess_input(img)

# Predict the class of the image
predictions = model.predict(np.array([img]))

# Get the class with the highest probability
predicted_class = labels[np.argmax(predictions)]

print(f"The image is predicted to belong to class: {predicted_class}")

1/1 [==============================] - 0s 19ms/step
The image is predicted to belong to class: B
